In [1]:
!mkdir -p ~/.cloudvolume/secrets
!cp /notebooks/*.json ~/.cloudvolume/secrets/
!pip install cloud-volume
!pip install scikit-image

    100% |################################| 71kB 2.2MB/s ta 0:00:011
    100% |################################| 81kB 3.5MB/s ta 0:00:011
    100% |################################| 419kB 2.3MB/s ta 0:00:01
    100% |################################| 92kB 4.5MB/s ta 0:00:011
    100% |################################| 71kB 5.4MB/s ta 0:00:011
    100% |################################| 133kB 4.2MB/s ta 0:00:01
    100% |################################| 92kB 5.4MB/s ta 0:00:011
    100% |################################| 194kB 3.6MB/s ta 0:00:01
    100% |################################| 7.1MB 212kB/s eta 0:00:01
    100% |################################| 51kB 12.1MB/s ta 0:00:01
    100% |################################| 143kB 9.8MB/s eta 0:00:01
    100% |################################| 143kB 9.8MB/s eta 0:00:01
    100% |################################| 266kB 5.5MB/s eta 0:00:01
    100% |################################| 61kB 11.7MB/s ta 0:00:01
    100% |####################

    100% |################################| 51kB 13.3MB/s ta 0:00:01
    100% |################################| 153kB 8.8MB/s eta 0:00:01
    100% |################################| 92kB 12.6MB/s ta 0:00:01
    100% |################################| 71kB 13.0MB/s ta 0:00:01
    100% |################################| 61kB 12.4MB/s ta 0:00:01
    100% |################################| 61kB 12.7MB/s ta 0:00:01
    100% |################################| 143kB 10.1MB/s ta 0:00:01
    100% |################################| 51kB 13.3MB/s ta 0:00:01
    100% |################################| 71kB 12.2MB/s ta 0:00:01
    100% |################################| 92kB 12.4MB/s ta 0:00:01
    100% |################################| 81kB 13.3MB/s ta 0:00:01
    100% |################################| 61kB 11.5MB/s ta 0:00:01
    100% |################################| 51kB 12.4MB/s ta 0:00:01
    100% |################################| 51kB 11.2MB/s ta 0:00:01
    100% |######################

    100% |################################| 51kB 12.2MB/s ta 0:00:01
    100% |################################| 92kB 12.5MB/s ta 0:00:01
    100% |################################| 81kB 12.6MB/s ta 0:00:01
    100% |################################| 552kB 2.9MB/s eta 0:00:01
    100% |################################| 51kB 11.8MB/s ta 0:00:01
    100% |################################| 102kB 12.0MB/s a 0:00:01
    100% |################################| 204kB 6.7MB/s eta 0:00:01
    100% |################################| 81kB 12.4MB/s ta 0:00:01
    100% |################################| 71kB 12.6MB/s ta 0:00:01
    100% |################################| 829kB 1.9MB/s eta 0:00:01
    100% |################################| 9.0MB 171kB/s eta 0:00:01
    100% |################################| 163kB 8.8MB/s eta 0:00:01


    100% |################################| 225kB 6.7MB/s eta 0:00:01
  Running setup.py bdist_wheel for posix-ipc ... done
  Stored in directory: /root/.cache/pip/wheels/5a/5c/17/d0e2d421abaf3b4097bf9db11108380d734195c6d15c24269d
  Running setup.py bdist_wheel for psutil ... done
  Stored in directory: /root/.cache/pip/wheels/fe/33/84/baea3d665de2d1af7e8f827f3883811bba5e4149443ccf8191
  Running setup.py bdist_wheel for tenacity ... done
  Stored in directory: /root/.cache/pip/wheels/cf/dd/89/99693d2e2c5954ccd7b7ba4fe9571b385712939d25e584a03a
  Running setup.py bdist_wheel for inflection ... done
  Stored in directory: /root/.cache/pip/wheels/9f/fa/4f/ef45a51543bd855e40494298824ad5e750109ad18fa80278ad
  Running setup.py bdist_wheel for Markdown ... done
  Stored in directory: /root/.cache/pip/wheels/34/2d/c2/75f4fd57c1b79e59f66388262ed37b69ceac77162988f8af63
  Running setup.py bdist_wheel for pandocfilters ... done
  Stored in directory: /root/.cache/pip/wheels/fc/a0/4c/ec26eaa3f225314

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |################################| 25.3MB 56kB/s  eta 0:00:01
    100% |################################| 1.6MB 835kB/s eta 0:00:01
    100% |################################| 634kB 2.5MB/s eta 0:00:01
    100% |################################| 5.7MB 277kB/s eta 0:00:01
    100% |################################| 51kB 10.9MB/s ta 0:00:01
  Running setup.py bdist_wheel for networkx ... done
  Stored in directory: /root/.cache/pip/wheels/44/c0/34/6f98693a554301bdb405f8d65d95bbcd3e50180cbfdd98a94e
  Running setup.py bdist_wheel for toolz ... done
  Stored in directory: /root/.cache/pip/wheels/f4/0c/f6/ce6b2d1aa459ee97cc3c0f82236302bd62d89c86c700219463
Successfully built networkx toolz
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [209]:
import os
import sys
import time
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import collections
import random

from cloudvolume import CloudVolume
from cloudvolume.lib import Vec, Bbox
from skimage.transform import resize

def invert_mask(mask, th=50/255.0, bias=0.2):
        bitmask = mask < th
        th_mask = bitmask * mask
        th_max = np.max(th_mask) + bias
        return (th_max - th_mask) / th_max

class Stack():
    """Collection of CloudVolume paths, with load & save operations
    """

    def __init__(self, paths):
        self.paths = paths

    def load(self, key, bbox, src_mip=0, dst_mip=0):
        src_cv = CloudVolume(self.paths[key], mip=src_mip, fill_missing=True)
        src_bbox = Bbox.from_slices(src_cv.slices_from_global_coords(bbox))
        img = src_cv[src_bbox.to_slices()][:,:,0,0]
        if dst_mip != src_mip:
                dst_cv = CloudVolume(self.paths[key], mip=dst_mip, fill_missing=True)
                dst_bbox = Bbox.from_slices(dst_cv.slices_from_global_coords(bbox))
                img = resize(img, dst_bbox.size3()[:2], preserve_range=True)
        return img

    def save(self, key, mip, bbox, img, is_field=False):
        """Save vector field to CloudVolume

        Args:
        * cv: CloudVolume object
        * mip: int representing mip-level of the CloudVolume
        * bbox: Bbox at mip 0 with size3 of (x,y,1)
        * img: nxm int64 ndarray with shape (x,y)
        * is_field: boolean whether img is a field object
                field object is 4d tensor with shape (1,x,y,2)
        """
        src_cv = CloudVolume(self.paths['image'], mip=mip)
        info = src_cv.info
        if is_field:
                info['data_type'] = 'float32'
                info['num_channels'] = 2
        cv = CloudVolume(self.paths[key], mip=mip, bounded=False, 
                                         info=info, fill_missing=True, 
                                         non_aligned_writes=True, cdn_cache=False)
        cv.commit_info()

        if is_field:
                s = cv.slices_from_global_coords(bbox) 
                s += (slice(0,2),)
                cv[s] = img.permute(1,2,0,3).data.cpu().numpy()
        else:
                s = cv.slices_from_global_coords(bbox)
                cv[s] = img[:,:,np.newaxis,np.newaxis]

class Optimizer():
    """

    Attributes:
        * src_image: nxm float64 ndarry normalized between [0,1]
            This is the image to be transformed by the returned vector field.
        * dst_images: list of nxm float64 ndarrays normalized between [0,1]
            This is the set of images that transformed src_image will be compared to.
        * src_smooth_mask: nxm float64 ndarray normalized between [0,1]
            The weight represents that degree to which a pixel participates in smooth
            deformation (0: not at all; 1: completely). This mask is used to reduce 
            the smoothness penalty for pixels that participate in a non-smooth 
            deformation.
            ### this mask is too severe on the smoothness penalty. consider changing
        * src_loss_mask: nxm float64 ndarray normalized between [0,1]
            This mask is transformed by the vector field and used to reduce the MSE
            for pixels that participate in a defect.
        * dst_masks_loss: list of nxm float64 ndarrays normalized between [0,1]
            Exactly like the src_mask_loss above. These masks are only used to reduce the 
            MSE for pixels that participate in a defect.
            ### unclear what `dst_masks_loss` is ???
    """

    def __init__(self, stack, bbox, dst_increment, image_mip, fold_mip, crack_mip, loss_mip, output_mip):
        self.stack = stack # the stack of images
        self.bbox = bbox # the "frame" in which to process
        self.image_mip = image_mip # lowest mip at which to process
        self.output_mip = output_mip # mip at which to produce the output image
        self.src_image = stack.load('image', bbox, src_mip=image_mip, dst_mip=image_mip) / 255.0
        self.src_fold = stack.load('fold', bbox, src_mip=fold_mip, dst_mip=image_mip) / 255.0
        self.src_crack = stack.load('crack', bbox, src_mip=crack_mip, dst_mip=image_mip) / 255.0
        self.src_smooth_mask = invert_mask(np.minimum(self.src_fold, self.src_crack))
        self.src_loss_mask = (stack.load('loss', bbox, src_mip=loss_mip, dst_mip=image_mip) == 0) * 1.0

        self.dst_images = []
        self.dst_masks = []
        for z in dst_increment: # run through the layers being compared/aligned to
            dst_bbox = self.bbox + [0,0,z]
            dst_image = stack.load('image', dst_bbox, src_mip=image_mip, dst_mip=image_mip)
            self.dst_images.append(dst_image)
            dst_mask = (stack.load('loss', dst_bbox, src_mip=loss_mip, dst_mip=image_mip) == 0) * 1.0
            self.dst_masks.append(dst_mask)
            if output_mip != image_mip:
                dst_image = stack.load('image', dst_bbox, src_mip=output_mip, dst_mip=output_mip)
            self.stack.save('output', output_mip, dst_bbox, dst_image, is_field=False)

        self.identities = {}
    
    # crops the image var by d/2 pixels on each edge of the dimensions specified by dims
    @staticmethod
    def center(var, dims, d):
        if not isinstance(d, collections.Sequence):
            d = [d for i in range(len(dims))]
        for idx, dim in enumerate(dims):
            if d[idx] == 0:
                continue
            var = var.narrow(dim, int(d[idx]/2), int(var.size()[dim] - d[idx]))
        return var
    
    # returns a grid containing the relative pixel location (unity under grid_sample())
    def get_identity_grid(self, dim, cache=True):
        if dim not in self.identities:
            gx, gy = np.linspace(-1, 1, dim), np.linspace(-1, 1, dim)
            I = np.stack(np.meshgrid(gx, gy))
            I = np.expand_dims(I, 0)
            I = torch.FloatTensor(I)
            I = torch.autograd.Variable(I, requires_grad=False)
            I = I.permute(0,2,3,1).cuda()
            self.identities[dim] = I
        if cache:
            return self.identities[dim]
        else:
            return self.identities[dim].clone()

    def jacob(self, fields, symmetric = True):
        # field (batch, x loc, y loc, component)
        def dx(f):
            p = Variable(torch.zeros((1,1,f.size(1),2))).cuda()
            if symmetric:
                return torch.cat((p, f[:,2:,:,:] - f[:,:-2,:,:], p), 1)
            else:
                return torch.cat((f[:,1:,:,:] - f[:,:-1,:,:], p), 1)
        def dy(f):
            p = Variable(torch.zeros((1,f.size(1),1,2))).cuda()
            if symmetric:
                return torch.cat((p, f[:,:,2:,:] - f[:,:,:-2,:], p), 2)
            else:
                return torch.cat((f[:,:,1:,:] - f[:,:,:-1,:], p), 2)
        fields = sum(map(lambda f: [dx(f), dy(f)], fields), [])
        field = torch.sum(torch.cat(fields, -1) ** 2, -1)
        return field

    def penalty(self, fields, mask=1):
        jacob = self.jacob(fields)
        jacob = torch.mul(jacob, mask)
        return jacob
#         return torch.sum(jacob)

    def render(self, field):
        ### why 255? Is 1.0 a valid value then? Why divide it at all and not just convert to float?
        src = stack.load('image', bbox, src_mip=self.output_mip, dst_mip=self.output_mip) / 255.0
        src = torch.FloatTensor(src).cuda()
        src = Variable(src).unsqueeze(0).unsqueeze(0) # unsqueeze adds a dimension to the tensor
        ### a bit cluttered and redundant since check done twice and no need for lambda func. clean up
        if self.output_mip < self.image_mip: # upsample the field if needed
            upsample = lambda x: nn.Upsample(scale_factor=2**x, mode='bilinear') if x >0 else (lambda y: y)
            field = upsample(self.image_mip - self.output_mip)(field.permute(0,3,1,2)).permute(0,2,3,1)
        y = F.grid_sample(src, field + self.get_identity_grid(field.size(2)))
        img = y.data.cpu().numpy()
        img = (img[0,0,:,:]*255).astype(np.uint8)
        self.stack.save('output', self.output_mip, self.bbox, img, is_field=False)

    def process(self, field=None, crop=0, write_field=True, ndownsamples=4, 
                                                                     currn=5, avgn=20, lambda1=0.4, lr=0.2, 
                                                                     eps=0.01, min_iter=20, max_iter=1000,
                                                                     write_penalty = True):
        """Compute vector field that minimizes mean squared error (MSE) between 
        transformed src_image & all dst_images regularized by the smoothness of the 
        vector field subject to masks that allow the vector field to not be smooth. 
        The minimization uses SGD.

        Args:
        * field: nxmx2 float64 torch tensor normalized between [0,1]
            This field represents the derived vector field that transforms the 
            src_image subject to the contraints of the minimization.
        * ndownsamples: int
            The number of mip layers to use in the alignment process
        * lr: float
            The learning rate at the lowest mip level (adjusted at higher mips)
        * currn and avgn: int
            The number of current and prior samples of the cost function to compare for stoppage
        * crop: int
            crops the returned vector field by this many pixels on each side

        Returns:
        * field: nxmx2 float64 torch tensor normalized between [0,1]
        """
        downsample = lambda x: nn.AvgPool2d(2**x,2**x, count_include_pad=False) if x > 0 else (lambda y: y)
        upsample = nn.Upsample(scale_factor=2, mode='bilinear')
        
        # convert images to PyTorch Variables
        s = torch.FloatTensor(self.src_image)
        # note:unsqueeze adds a dimension to the tensor
        src_var = Variable((s - torch.mean(s)) / torch.std(s)).cuda().unsqueeze(0).unsqueeze(0)
        src_smooth_mask_var = Variable(torch.FloatTensor(self.src_smooth_mask)).cuda().unsqueeze(0).unsqueeze(0)
        src_loss_mask_var = Variable(torch.FloatTensor(self.src_loss_mask)).cuda().unsqueeze(0).unsqueeze(0)
        dst_vars = []
        dst_mask_vars = []
        for d, m in zip(self.dst_images, self.dst_masks):
            d = torch.FloatTensor(d)
            dst_var = Variable((d - torch.mean(d)) / torch.std(d)).cuda().unsqueeze(0).unsqueeze(0)
            dst_vars.append(dst_var)
            mask_var = Variable(torch.FloatTensor(m)).cuda().unsqueeze(0).unsqueeze(0)
            dst_mask_vars.append(mask_var)
        
        # initialize the field
        dim = int(src_var.size()[-1] / (2 ** (ndownsamples - 1)))
        if field is None:
            field = Variable(torch.zeros((1,dim,dim,2))).cuda().detach()
        else:
            field = Variable(field).cuda().detach()
        field.requires_grad = True
        
        updates = 0
        for k in reversed(range(ndownsamples)): # mip levels (image_mip+ndownsamples-1) through image_mip
            src_ = downsample(k)(src_var).detach()
            src_.requires_grad = False    ### isn't this redundant? detach already sets this
            src_smooth_mask_ = downsample(k)(src_smooth_mask_var).detach()
            src_smooth_mask_.requires_grad = False
            src_loss_mask_ = downsample(k)(src_loss_mask_var).detach()
            src_loss_mask_.requires_grad = False
            dst_list_, dst_mask_list_ = [], []
            for d, m in zip(dst_vars, dst_mask_vars):
                dst_ = downsample(k)(d).detach()
                dst_.requires_grad = False
                dst_list_.append(dst_)
                mask_ = downsample(k)(m).detach()
                mask_.requires_grad = False
                dst_mask_list_.append(mask_)
            field = field.detach() ### why, if already detached above?
            field.requires_grad = True # not redundant (detach makes it False by default)
            opt = torch.optim.SGD([field], lr=lr/(k+1)) ### why is the learning rate increased for lower mips??
            #sched = lr_scheduler.StepLR(opt, step_size=1, gamma=0.995)
            costs = [] # list of decreasing costs over time
            start_updates = updates ### not used (?)
            print('Downsample factor {0}x'.format(2**k))
            while True:
                updates += 1
                I = self.get_identity_grid(field.size(2))
                pred = F.grid_sample(src_, field + I)
                pred_loss_mask = F.grid_sample(src_loss_mask_, field + I)
                pred_smooth_mask = F.grid_sample(src_smooth_mask_, field + I)
                ### why is 128 hardcoded?
                # squeeze removes a dimension from the tensor
                centered_mask = self.center(pred_smooth_mask.squeeze(0), (1,2), 128/(2**k))
                centered_field = self.center(field, (1,2), 128/(2**k))
                # calaculate the smoothness penalty
                ### currently the masks are transformed with the field, so if the field is wrong,
                ### so is the mask (!)
                penalty_error_image = self.penalty([centered_field], centered_mask)
                penalty1 = torch.sum(penalty_error_image)
                cost = penalty1 * lambda1/(k+1) ### why divide by (k+1) ?
                for d, m in zip(dst_list_, dst_mask_list_):
                    # merge the pixel-perfect mask from the neighboring slice with that of the current slice
                    mask = torch.mul(pred_loss_mask, m) # all 0.0 or 1.0:   0 := ignore
                    mse = torch.mul(pred - d, mask)**2 # mse with the neighboring slice
                    ### why -1,-2? what are the dimensions of mse?
                    mse_error_image = self.center(mse, (-1,-2), 128 / (2**k))
                    cost += torch.mean(mse_error_image)
                    print ("penalty_error_image: ", penalty_error_image.size())
                    print ("mse_error_image: ", mse_error_image.size())
#                     penalty_error_image += mse_error_image
                #cost = diff + penalty1 * lambda1/(k+1)
                print(cost.data.cpu().numpy())
                costs.append(cost)
                cost.backward()
                opt.step()
                #sched.step()uniform
                opt.zero_grad()
                if len(costs) > avgn + currn and len(costs)>min_iter:
                    ### can already break here if len(costs)>max_iter
                    hist_costs = costs[-(avgn+currn):-currn]
                    hist = sum(hist_costs).data[0] / avgn
                    curr = sum(costs[-currn:]).data[0] / currn
                    ### why does the epsilon increase for lower mip levels? Is that because of larger cost?
                    if abs((hist-curr)/hist) < eps/(2**k) or len(costs)>max_iter:
                        ### may be useful to know which of these cases happens. maybe print
                        break
                #print (downsamples, updates - start_updates)
            if write_field:
                self.stack.save('field', self.image_mip+k, self.bbox, field + I, is_field=True)
            if write_penalty:
                # output the smoothness penalty
                print("before:", penalty_error_image.size())
                img = penalty_error_image.squeeze(0).data.cpu().numpy()
                img_max = np.max(img) # max value to rescale the image
                print("smoothness penalty img_max:", img_max)
                img = (img*255/img_max).astype(np.uint8)
                v = Vec(64*(2**image_mip),64*(2**image_mip),0)
                bbox = Bbox(self.bbox.minpt + v, self.bbox.maxpt - v)
                print ("self:", self.bbox)
                print ("modified:", bbox)
                self.stack.save('penalty', self.image_mip+k, bbox, img, is_field=False)
                # output the mse
                img = mse_error_image.squeeze(0).squeeze(0).data.cpu().numpy()
                img_max = np.max(img) # max value to rescale the image
                print("mse img_max:", img_max)
                img = (img*255/img_max).astype(np.uint8)
                v = Vec(64*(2**image_mip),64*(2**image_mip),0)
                bbox = Bbox(self.bbox.minpt + v, self.bbox.maxpt - v)
                self.stack.save('mse', self.image_mip+k, bbox, img, is_field=False)
            if k > 0:
                field = upsample(field.permute(0,3,1,2)).permute(0,2,3,1)
        #print(cost.data[0], diff.data[0], penalty1.data[0])
        print('done:', updates)
        print(field.shape)
        # return self.center(field, (1,2), crop*2).data.cpu().numpy()[0]
        return self.center(field, (1,2), crop*2)

In [164]:
name = 'optimizertestsbn'
paths = {
    'image': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked',
    'fold': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1',
    'crack': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3',
    'loss': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/ground_truth',
    'output': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests_bn/{0}'.format(name),
    'field': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests_bn/{0}/field'.format(name),
    'penalty': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests_bn/{0}/penalty'.format(name),
    'mse': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests_bn/{0}/mse'.format(name),
}
stack = Stack(paths)
offsets = [
              Vec(102716, 107077, 526), # fold
              Vec(102320, 111106, 526), # crack
              Vec(102565, 103904, 527), # no defect
              Vec(104799, 109774, 526) # test (remove later)
          ]
size = Vec(2048, 2048, 1)
image_mip = 2
fold_mip = 5
crack_mip = 5
loss_mip = 2
output_mip = 1
dst_increment = [-1,1]
for offset in offsets:
    bbox = Bbox(offset, offset+size)
    o = Optimizer(stack, bbox, dst_increment, image_mip, fold_mip, crack_mip, loss_mip, output_mip)
    field = o.process(ndownsamples=5, currn=5, avgn=20, lambda1=0.9, lr=0.1, 
                                                  eps=0.001, min_iter=100, max_iter=5000)
    o.render(field)

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
Downloading:   0%|          | 0/4 [00:00<?, ?it/s]
Downloading: 6it [00:00, 31.09it/s]               
Downloading:   0%|          | 0/4 [00:00<?, ?it/s]
Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]


Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]



Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  9

Downsample factor 16x
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
[0.26509356]
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
[0.25215346]
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
[0.24382745]
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
[0.23793799]
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
[0.23314849]
pen

penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
[0.14653161]
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
[0.14636311]
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
[0.14621928]
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
[0.14605404]
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
[0.14594698]
penalty_error_image:  tor

penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
[0.14235163]
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
[0.14236026]
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
[0.14229728]
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
[0.14225781]
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
[0.14225748]
penalty_error_image:  tor

penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
[0.14075607]
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
[0.14076011]
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
[0.14074394]
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
[0.14073841]
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
penalty_error_image:  torch.Size([1, 24, 24])
mse_error_image:  torch.Size([1, 1, 24, 24])
[0.14070009]
penalty_error_image:  tor

...

Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  5.34it/s]



mse img_max: 15.27688


Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  6.59it/s]



done: 1275
torch.Size([1, 512, 512, 2])


Downloading:   0%|          | 0/4 [00:00<?, ?it/s]
Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]


Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]



Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  6.50it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  7.18it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  7.17it/s]



In [98]:
from scipy import ndimage
ndimage.filters.convolve

<function scipy.ndimage.filters.convolve(input, weights, output=None, mode='reflect', cval=0.0, origin=0)>

In [101]:
np.concatenate

<function numpy.core.multiarray.concatenate>

In [122]:
kernel = np.array([[200 - (i-10)**2 - (j-10)**2 for j in range(21)] for i in range(21)])

In [117]:
np.stack([np.concatenate([np.arange(x,x+10), np.arange(x+10,x-1,-1)]) for x ain range(10)] + [np.concatenate([np.arange(x,x+10), np.arange(x+10,x-1,-1)]) for x in range(10,-1,-1)])

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  9,  8,  7,  6,  5,
         4,  3,  2,  1,  0],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 10,  9,  8,  7,  6,
         5,  4,  3,  2,  1],
       [ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 11, 10,  9,  8,  7,
         6,  5,  4,  3,  2],
       [ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 12, 11, 10,  9,  8,
         7,  6,  5,  4,  3],
       [ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 13, 12, 11, 10,  9,
         8,  7,  6,  5,  4],
       [ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 14, 13, 12, 11, 10,
         9,  8,  7,  6,  5],
       [ 6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 15, 14, 13, 12, 11,
        10,  9,  8,  7,  6],
       [ 7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 16, 15, 14, 13, 12,
        11, 10,  9,  8,  7],
       [ 8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 17, 16, 15, 14, 13,
        12, 11, 10,  9,  8],
       [ 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 18, 17, 16, 15, 14,
        13, 12, 11,

In [138]:
kernel_size = 61
kernel = np.array([[(kernel_size-1)**2/2 - (i-kernel_size//2)**2 - (j-kernel_size//2)**2 for j in range(kernel_size)] for i in range(kernel_size)])
kernel = kernel/np.max(kernel)*255
kernel

array([[ 0.        ,  8.35833333, 16.43333333, ..., 16.43333333,
         8.35833333,  0.        ],
       [ 8.35833333, 16.71666667, 24.79166667, ..., 24.79166667,
        16.71666667,  8.35833333],
       [16.43333333, 24.79166667, 32.86666667, ..., 32.86666667,
        24.79166667, 16.43333333],
       ...,
       [16.43333333, 24.79166667, 32.86666667, ..., 32.86666667,
        24.79166667, 16.43333333],
       [ 8.35833333, 16.71666667, 24.79166667, ..., 24.79166667,
        16.71666667,  8.35833333],
       [ 0.        ,  8.35833333, 16.43333333, ..., 16.43333333,
         8.35833333,  0.        ]])

In [140]:
# widen pixel-perfect mask
name = 'optimizertestsbn'
paths = {
    'image': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked',
    'fold': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1',
    'crack': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3',
    'loss': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/ground_truth',
    'output': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests_bn/{0}'.format(name),
    'field': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests_bn/{0}/field'.format(name),
    'penalty': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests_bn/{0}/penalty'.format(name),
    'mse': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests_bn/{0}/mse'.format(name),
    'widened': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests_bn/{0}/widened'.format(name),
}
stack = Stack(paths)
offsets = [
              Vec(102716, 107077, 526), # fold
              Vec(102320, 111106, 526), # crack
              Vec(102565, 103904, 527), # no defect
              Vec(104799, 109774, 526) # test (remove later)
          ]
size = Vec(2048, 2048, 1)
image_mip = 2
fold_mip = 5
crack_mip = 5
loss_mip = 2
output_mip = 1
dst_increment = [-1,1]

for offset in offsets:
    bbox = Bbox(offset, offset+size)
    pixelperfect_mask = stack.load('loss', bbox, src_mip=loss_mip, dst_mip=image_mip).astype(np.float64)

    widened = ndimage.filters.convolve(pixelperfect_mask, kernel, mode='constant', cval=0.0)
    widened = (widened / (np.max(widened)+0.0) * 255).astype(np.uint8)
    stack.save('widened', loss_mip, bbox, widened, is_field=False)

Downloading:   0%|          | 0/4 [00:00<?, ?it/s]
Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]


Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]



Rechunking image: 100%|██████████| 1/1 [00:00<00:00, 10.56it/s]


Rechunking image: 100%|██████████| 1/1 [00:00<00:00, 12.18it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  8.84it/s]


Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  6.81it/s]

Downloading:   0%|          | 0/4 [00:00<?, ?it/s]
Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]


Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]



Downloading:   0%|          | 0/2 [00:00<?, ?it/s]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encounte

In [53]:
nn.ZeroPad2d

torch.nn.modules.padding.ZeroPad2d